# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Trade.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 WalletBalancer

This notebook deals with balancing a wallet after processing liquidations, so that it has appropriate funds for the next liquidation.

We want to be able to maintain liquidity in our wallet. For instance if there are a lot of ETH shorts being liquidated, we'll need to supply ETH, but what do we do when we run out of ETH and there are still liquidations to perform?

We 'balance' our wallet tokens, buying or selling or swapping them as required.

In [ ]:
import abc
import logging
import typing

from decimal import Decimal
from solana.publickey import PublicKey

from BaseModel import BasketTokenMetadata, Group, TokenValue
from Context import Context
from TradeExecutor import TradeExecutor
from Wallet import Wallet


# Target Balances

To be able to maintain the right balance of tokens, we need to know what the right balance is. Different people have different opinions, and we don't all have the same value in our liquidator accounts, so we need a way to allow whoever is running the liquidator to specify what the target token balances should be.

There are two possible approaches to specifying the target value:
* A 'fixed' value, like 10 ETH
* A 'percentage' value, like 20% ETH

Percentage is trickier, because to come up with the actual target we need to take into account the wallet value and the current price of the target token.

The way this all hangs together is:
* A parser parses string values (probably from a command-line) into `TargetBalance` objects.
* There are two types of `TargetBalance` objects - `FixedTargetBalance` and `PercentageTargetBalance`.
* To get the actual `TokenValue` for balancing, the `TargetBalance` must be 'resolved' by calling `resolve()` with the appropriate token price and wallet value.

## TargetBalance class

This is the abstract base class for our target balances, to allow them to be treated polymorphically.

In [ ]:
class TargetBalance(metaclass=abc.ABCMeta):
    def __init__(self, token: BasketTokenMetadata):
        self._token = token

    @property
    def token(self) -> BasketTokenMetadata:
        return self._token

    @abc.abstractmethod
    def resolve(self, current_price: Decimal, total_value: Decimal) -> TokenValue:
        raise NotImplementedError("TargetBalance.resolve() is not implemented on the base type.")

    def __repr__(self) -> str:
        return f"{self}"



## FixedTargetBalance class

This is the simple case, where the `FixedTargetBalance` object contains enough information on its own to build the resolved `TokenValue` object.

In [ ]:
class FixedTargetBalance(TargetBalance):
    def __init__(self, token: BasketTokenMetadata, value: Decimal):
        super().__init__(token)
        self.value = value

    def resolve(self, current_price: Decimal, total_value: Decimal) -> TokenValue:
        return TokenValue(self.token, self.value)

    def __str__(self) -> str:
        return f"""« FixedTargetBalance [{self.value} {self.token.name}] »"""


## PercentageTargetBalance

This is the more complex case, where the target is a percentage of the total wallet balance.

So, to actually calculate the right target, we need to know the total wallet balance and the current price. Once we have those the calculation is just:
> _wallet fraction_ is _percentage_ of _wallet value_

> _target balance_ is _wallet fraction_ divided by _token price_

In [ ]:
class PercentageTargetBalance(TargetBalance):
    def __init__(self, token: BasketTokenMetadata, target_percentage: Decimal):
        super().__init__(token)
        self.target_fraction = target_percentage / 100

    def resolve(self, current_price: Decimal, total_value: Decimal) -> TokenValue:
        target_value = total_value * self.target_fraction
        target_size = target_value / current_price
        return TokenValue(self.token, target_size)

    def __str__(self) -> str:
        return f"""« PercentageTargetBalance [{self.target_fraction * 100}% {self.token.name}] »"""


## TargetBalanceParser class

The `TargetBalanceParser` takes a string like "BTC:0.2" or "ETH:20%" and returns the appropriate TargetBalance object.

This has a lot of manual error handling because it's likely the error messages will be seen by people and so we want to be as clear as we can what specifically is wrong.

In [ ]:
class TargetBalanceParser:
    def __init__(self, tokens: typing.List[BasketTokenMetadata]):
        self.tokens = tokens

    def parse(self, to_parse: str):
        try:
            token_name, value = to_parse.split(":")
        except Exception as exception:
            raise Exception(f"Could not parse target balance '{to_parse}'") from exception

        matching_tokens = list([t for t in self.tokens if t.name == token_name.upper()])
        if len(matching_tokens) == 0:
            raise Exception(f"Token '{token_name}' not found in tokens {self.tokens}")

        assert(len(matching_tokens) == 1)
        token = matching_tokens[0]

        # The value we have may be an int (like 27), a fraction (like 0.1) or a percentage
        # (like 25%). In all cases we want the number as a number, but we also want to know if
        # we have a percent or not
        values = value.split("%")
        numeric_value_string = values[0]
        try:
            numeric_value = Decimal(numeric_value_string)
        except Exception as exception:
            raise Exception(f"Could not parse '{numeric_value_string}' as a decimal number. It should be formatted as a decimal number, e.g. '2.345', with no surrounding spaces.") from exception

        if len(values) > 2:
            raise Exception(f"Could not parse '{value}' as a decimal percentage. It should be formatted as a decimal number followed by a percentage sign, e.g. '30%', with no surrounding spaces.")

        if len(values) == 1:
            return FixedTargetBalance(token, numeric_value)
        else:
            return PercentageTargetBalance(token, numeric_value)



# sort_changes_for_trades function

It's important to process SELLs first, so we have enough funds in the quote balance for the BUYs.

It looks like this function takes size into account, but it doesn't really - 2 ETH is smaller than 1 BTC (for now?) but the value 2 will be treated as bigger than 1. We don't really care that much as long as we have SELLs before BUYs. (We could, later, take price into account for this sorting but we don't need to now so we don't.)

In [ ]:
def sort_changes_for_trades(changes: typing.List[TokenValue]) -> typing.List[TokenValue]:
    return sorted(changes, key=lambda change: change.value)


# calculate_required_balance_changes function

Takes a list of current balances, and a list of desired balances, and returns the list of changes required to get us to the desired balances.

In [ ]:
def calculate_required_balance_changes(current_balances: typing.List[TokenValue], desired_balances: typing.List[TokenValue]) -> typing.List[TokenValue]:
    changes: typing.List[TokenValue] = []
    for desired in desired_balances:
        current = [bal for bal in current_balances if bal.token.mint == desired.token.mint][0]
        change = TokenValue(desired.token, desired.value - current.value)
        changes += [change]

    return changes


# FilterSmallChanges class

Allows us to filter out changes that aren't worth the effort.

For instance, if our desired balance requires changing less than 1% of our total balance, it may not be worth bothering with right not.

Calculations are based on the total wallet balance, rather than the magnitude of the change per-token, because a change of 0.01 of one token may be worth more than a change of 10 in another token. Normalising values to our wallet balance makes these changes easier to reason about.

In [ ]:
class FilterSmallChanges:
    def __init__(self, action_threshold: Decimal, balances: typing.List[TokenValue], prices: typing.List[TokenValue]):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.prices: typing.Dict[str, TokenValue] = {}
        total = Decimal(0)
        for balance in balances:
            price = [p for p in prices if p.token.mint == balance.token.mint][0]
            self.prices[price.token.mint.to_base58()] = price
            total += price.value * balance.value
        self.total_balance = total
        self.action_threshold_value = total * action_threshold
        self.logger.info(f"Wallet total balance of {total} gives action threshold value of {self.action_threshold_value}")


    def allow(self, token_value: TokenValue) -> bool:
        price = self.prices[token_value.token.mint.to_base58()]
        value = price.value * token_value.value
        absolute_value = value.copy_abs()
        result = absolute_value > self.action_threshold_value

        self.logger.info(f"Value of {token_value.token.name} trade is {absolute_value}, threshold value is {self.action_threshold_value}. Is this worth doing? {result}.")
        return result


# WalletBalancers

We want two types of this class:
* A 'null' implementation that adheres to the interface but doesn't do anything, and
* A 'live' implementation that actually does the balancing.

This allows us to have code that implements logic including wallet balancing, without having to worry about whether the user wants to re-balance or not - we can just plug in the 'null' variant and the logic all still works.

To have this work we define an abstract base class `WalletBalancer` which defines the interface, then a `NullWalletBalancer` which adheres to this interface but doesn't perform any action, and finally the real `LiveWalletBalancer` which can perform the balancing action.

## WalletBalancer class

This is the abstract class which defines the interface.

In [ ]:
class WalletBalancer(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def balance(self, prices: typing.List[TokenValue]):
        raise NotImplementedError("WalletBalancer.balance() is not implemented on the base type.")


## NullWalletBalancer class

This is the 'empty', 'no-op', 'dry run' wallet balancer which doesn't do anything but which can be plugged into algorithms that may want balancing logic.

In [ ]:
class NullWalletBalancer(WalletBalancer):
    def balance(self, prices: typing.List[TokenValue]):
        pass


## LiveWalletBalancer class

This is the high-level class that does much of the work.

In [ ]:
class LiveWalletBalancer(WalletBalancer):
    def __init__(self, context: Context, wallet: Wallet, trade_executor: TradeExecutor, action_threshold: Decimal, tokens: typing.List[BasketTokenMetadata], target_balances: typing.List[TokenValue]):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.context: Context = context
        self.wallet: Wallet = wallet
        self.trade_executor: TradeExecutor = trade_executor
        self.action_threshold: Decimal = action_threshold
        self.tokens: typing.List[BasketTokenMetadata] = tokens
        self.target_balances: typing.List[TokenValue] = target_balances

    def balance(self, prices: typing.List[TokenValue]):
        padding = "\n    "
        def balances_report(balances) -> str:
            return padding.join(list([f"{bal}" for bal in balances]))
        current_balances = self._fetch_balances()
        self.logger.info(f"Starting balances: {padding}{balances_report(current_balances)}")
        balance_changes = calculate_required_balance_changes(current_balances, self.target_balances)
        self.logger.info(f"Full balance changes: {padding}{balances_report(balance_changes)}")

        dont_bother = FilterSmallChanges(self.action_threshold, current_balances, prices)
        filtered_changes = list(filter(dont_bother.allow, balance_changes))
        self.logger.info(f"Filtered balance changes: {padding}{balances_report(filtered_changes)}")
        if len(filtered_changes) == 0:
            self.logger.info("No balance changes to make.")
            return

        sorted_changes = sort_changes_for_trades(filtered_changes)
        self._make_changes(sorted_changes)
        updated_balances = self._fetch_balances()
        self.logger.info(f"Finishing balances: {padding}{balances_report(updated_balances)}")

    def _make_changes(self, balance_changes: typing.List[TokenValue]):
        self.logger.info(f"Balance changes to make: {balance_changes}")
        for change in balance_changes:
            if change.value < 0:
                self.trade_executor.sell(change.token.name, change.value.copy_abs())
            else:
                self.trade_executor.buy(change.token.name, change.value.copy_abs())

    def _fetch_balances(self) -> typing.List[TokenValue]:
        balances: typing.List[TokenValue] = []
        for token in self.tokens:
            balance = TokenValue(token, self.context.fetch_token_balance(self.wallet.address, token.mint))
            balances += [balance]

        return balances



# Testing

In [ ]:
def _notebook_tests():
    log_level = logging.getLogger().level
    try:
        logging.getLogger().setLevel(logging.CRITICAL)
        eth = BasketTokenMetadata("ETH", PublicKey("2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk"), Decimal(6), None, None)
        btc = BasketTokenMetadata("BTC", PublicKey("9n4nbM75f5Ui33ZbPYXn59EwSgE8CGsHtAeTH5YFeJ9E"), Decimal(6), None, None)
        usdt = BasketTokenMetadata("USDT", PublicKey("Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB"), Decimal(6), None, None)
        current_prices = [
            TokenValue(eth, Decimal("4000")),
            TokenValue(btc, Decimal("60000")),
            TokenValue(usdt, Decimal("1")),
        ]
        current_balances = [
            TokenValue(eth, Decimal("0.5")),
            TokenValue(btc, Decimal("0.2")),
            TokenValue(usdt, Decimal("10000")),
        ]
        desired_balances = [
            TokenValue(eth, Decimal("1")),
            TokenValue(btc, Decimal("0.1"))
        ]

        changes = calculate_required_balance_changes(current_balances, desired_balances)

        assert(changes[0].token.name == "ETH")
        assert(changes[0].value == Decimal("0.5"))
        assert(changes[1].token.name == "BTC")
        assert(changes[1].value == Decimal("-0.1"))

        parsed_balance_change = FixedTargetBalance(eth, Decimal("0.1"))
        assert(parsed_balance_change.token == eth)
        assert(parsed_balance_change.value == Decimal("0.1"))

        percentage_parsed_balance_change = PercentageTargetBalance(eth, Decimal(33))
        assert(percentage_parsed_balance_change.token == eth)

        current_eth_price = Decimal(2000) # It's $2,000 per ETH
        current_account_value = Decimal(10000) # We hold $10,000 in total across all assets in our account.
        resolved_parsed_balance_change = percentage_parsed_balance_change.resolve(current_eth_price, current_account_value)
        assert(resolved_parsed_balance_change.token == eth)
        # 33% of $10,000 is $3,300
        # $3,300 spent on ETH gives us 1.65 ETH
        assert(resolved_parsed_balance_change.value == Decimal("1.65"))

        parser = TargetBalanceParser([eth, btc, usdt])
        parsed_percent = parser.parse("eth:10%")
        assert(parsed_percent.token == eth)
        assert(parsed_percent.target_fraction == Decimal("0.1"))

        parsed_fixed = parser.parse("eth:70")
        assert(parsed_fixed.token == eth)
        assert(parsed_fixed.value == Decimal(70))

        action_threshold = Decimal("0.01") # Don't bother if it's less than 1% of the total value (24,000)
        dont_bother = FilterSmallChanges(action_threshold, current_balances, current_prices)

        # 0.05 ETH is worth $200 at our test prices, which is less than our $240 threshold
        assert(not dont_bother.allow(TokenValue(eth, Decimal("0.05"))))

        # 0.05 BTC is worth $3,000 at our test prices, which is much more than our $240 threshold
        assert(dont_bother.allow(TokenValue(btc, Decimal("0.05"))))

        eth_buy = TokenValue(eth, Decimal("5"))
        btc_sell = TokenValue(btc, Decimal("-1"))
        sorted_changes = sort_changes_for_trades([
            eth_buy,
            btc_sell
        ])

        assert(sorted_changes[0] == btc_sell)
        assert(sorted_changes[1] == eth_buy)
    finally:
        logging.getLogger().setLevel(log_level)

_notebook_tests()
del _notebook_tests

# 🏃 Running

If running interactively, try to buy then sell 0.1 ETH.


In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    from Context import default_context

    group = Group.load(default_context)
    eth = group.tokens[group.index_of_token_by_name("eth")]
    btc = group.tokens[group.index_of_token_by_name("btc")]
    usdt = group.tokens[group.index_of_token_by_name("usdt")]

    parser = TargetBalanceParser([eth, btc])
    eth_target = parser.parse("ETH:20%")
    btc_target = parser.parse("btc:0.05")
    prices = [Decimal("60000"), Decimal("4000"), Decimal("1")] # Ordered as per Group index ordering
    desired_balances = []
    for target in [eth_target, btc_target]:
        token_index = group.index_of_token(target.token)
        price = prices[token_index]
        resolved = target.resolve(price, Decimal(10000))
        desired_balances += [resolved]

    assert(desired_balances[0].token.name == "ETH")
    assert(desired_balances[0].value == Decimal("0.5"))
    assert(desired_balances[1].token.name == "BTC")
    assert(desired_balances[1].value == Decimal("0.05"))

    current_balances = [
        TokenValue(eth, Decimal("0.6")), # Worth $2,400 at the test prices
        TokenValue(btc, Decimal("0.01")), # Worth $6,00 at the test prices
        TokenValue(usdt, Decimal("7000")), # Remainder of $10,000 minus above token values
    ]

    changes = calculate_required_balance_changes(current_balances, desired_balances)
    for change in changes:
        order_type = "BUY" if change.value > 0 else "SELL"
        print(f"{change.token.name} {order_type} {change.value}")

    # To get from our current balances of 0.6 ETH and 0.01 BTC to our desired balances of
    # 0.5 ETH and 0.05 BTC, we need to sell 0.1 ETH and buy 0.04 BTC. But we want to do the sell
    # first, to make sure we have the proper liquidity when it comes to buying.
    sorted_changes = sort_changes_for_trades(changes)
    assert(sorted_changes[0].token.name == "ETH")
    assert(sorted_changes[0].value == Decimal("-0.1"))
    assert(sorted_changes[1].token.name == "BTC")
    assert(sorted_changes[1].value == Decimal("0.04"))
